In [2]:
import pandas as pd
from datetime import datetime as dt
import datetime as dat
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
jobs = pd.read_csv("../data/fullsample.csv", nrows=100000)

In [4]:
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  nrows=1000)

In [5]:
ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  nrows=1000
                  )

In [6]:
colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
ce5.columns=colnames
ce6.columns=colnames

In [7]:
ce5['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce5['END']]
ce5['TIME'] = ce5['TIME'].str.replace('time', '', regex=False).astype(float)
ce5['USER'] = ce5['USER'].str.replace('user', '', regex=False).astype(int)
ce5['RETRY'] = ce5['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce5['RETURNCODE'] = ce5['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce5['COMMAND'] = ce5['COMMAND'].str.replace('command', '', regex=False)

In [8]:
ce6['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce6['END']]
ce6['TIME'] = ce6['TIME'].str.replace('time', '', regex=False).astype(float)
ce6['USER'] = ce6['USER'].str.replace('user', '', regex=False).astype(int)
ce6['RETRY'] = ce6['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce6['RETURNCODE'] = ce6['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce6['COMMAND'] = ce6['COMMAND'].str.replace('command', '', regex=False)

In [9]:
sbatch = ce6.loc[ce6['COMMAND'].str.contains('/usr/bin/sbatch')]

In [10]:
jobs = jobs[jobs['END'] != 'Unknown']

In [11]:
jobs['END'] = [dt.strptime(row.replace('T', ' ') + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in jobs['END']]
# Fixes jobs BEGIN and END columns to datetime
jobs['BEGIN'] = [dt.strptime(row.replace('T', ' ') + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in jobs['BEGIN']]

In [12]:
jobs['USEDMEM'] = jobs['USEDMEM'].str[:-1]
# Fixes USEDMEM column to remove the M and convert to a numeric
jobs['USEDMEM'] = pd.to_numeric(jobs['USEDMEM'])

In [13]:
# Cleans memory of nodes/CPUs to allow for numeric manipulation
jobs.insert(5, 'Mc_Mn', jobs['REQMEM'].str[-2:])
jobs['REQMEM']  = jobs['REQMEM'].str[:-2]
jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])

In [14]:
diffmem = jobs['REQMEM'] - jobs['USEDMEM']

In [15]:
jobs.insert(7, 'DIFFMEM', diffmem)

In [16]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99993 entries, 1 to 99999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   JOBID      99993 non-null  object        
 1   STATE      99993 non-null  object        
 2   BEGIN      99993 non-null  datetime64[ns]
 3   END        99993 non-null  datetime64[ns]
 4   REQMEM     99993 non-null  int64         
 5   Mc_Mn      99993 non-null  object        
 6   USEDMEM    88491 non-null  float64       
 7   DIFFMEM    88491 non-null  float64       
 8   REQTIME    99993 non-null  object        
 9   USEDTIME   99993 non-null  object        
 10  NODES      99993 non-null  int64         
 11  CPUS       99993 non-null  int64         
 12  PARTITION  99993 non-null  object        
 13  EXITCODE   99993 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(3), object(7)
memory usage: 11.4+ MB


In [17]:
USEDMEM_pivot = jobs.pivot_table(values='USEDMEM', index=['NODES','EXITCODE', 'STATE'], aggfunc='mean')

In [18]:
REQMEM_pivot = jobs.pivot_table(values='REQMEM', index=['Mc_Mn','EXITCODE', 'STATE'], aggfunc='mean')

The **EXITCODE** gives the [exit code](https://www.agileconnection.com/article/overview-linux-exit-codes) for the job, with "0:0" indicating a successful job. Exit codes have two numbers, where if the first number is non-zero, it indicates a problem on the server side and if the second is nonzero, it indicates a problem on the user side.

In [52]:
jobs.groupby(['STATE', 'PARTITION', 'EXITCODE'])['DIFFMEM'].describe()

count           mean  \
STATE               PARTITION      EXITCODE                           
CANCELLED           pascal         0:0           0.0            NaN   
                                   0:9          10.0   32311.875000   
                    production     0:0           0.0            NaN   
                                   0:15          3.0   83917.410000   
                                   0:9           5.0    7220.980000   
                                   1:0           0.0            NaN   
                    turing         0:15          2.0   39520.295000   
                                   0:9           7.0   18149.742857   
CANCELLED by 200557 production     0:9           1.0  204495.570000   
CANCELLED by 505355 turing         0:9           1.0   17968.820000   
COMPLETED           cgw-capra1     0:0           6.0   69224.521667   
                    cgw-dougherty1 0:0           4.0   74570.282500   
                    cgw-maizie     0:0          14.0   92485.076429   
                    cgw-platypus   0:0          10.0   99704.292000   
                    cgw-tbi01      0:0           1.0  147246.230000   
                    debug          0:0           1.0    2039.920000   
                    maxwell        0:0           3.0   42009.006667   
                    nogpfs         0:0         775.0   67602.265252   
                    pascal         0:0        1127.0    8395.336957   
                    production     0:0       86026.0    5173.768696   
                    sam            0:0         425.0    1925.240824   
                    turing         0:0          13.0   22914.070769   
FAILED              maxwell        1:0           2.0   23767.775000   
                    pascal         1:0           1.0   15570.590000   
                                   2:0           1.0   32760.730000   
                    production     121:0         2.0   12206.475000   
                                   1:0          19.0    8357.795789   
                                   2:0           2.0  131062.890000   
                                   3:0           1.0   32760.880000   
                                   6:0           2.0  131050.095000   
                    turing         1:0           1.0   40953.940000   
NODE_FAIL           cgw-dougherty1 1:0           0.0            NaN   
OUT_OF_MEMORY       production     0:125        26.0    -199.681154   

                                                       std        min  \
STATE               PARTITION      EXITCODE                             
CANCELLED           pascal         0:0                 NaN        NaN   
                                   0:9         1039.620269   29606.54   
                    production     0:0                 NaN        NaN   
                                   0:15       40744.103915   60306.25   
                                   0:9         5956.790771     939.66   
                                   1:0                 NaN        NaN   
                    turing         0:15           1.760696   39519.05   
                                   0:9        14584.013364    3613.28   
CANCELLED by 200557 production     0:9                 NaN  204495.57   
CANCELLED by 505355 turing         0:9                 NaN   17968.82   
COMPLETED           cgw-capra1     0:0        85010.873749     796.89   
                    cgw-dougherty1 0:0        27319.436468   34550.81   
                    cgw-maizie     0:0        48370.401796    2857.34   
                    cgw-platypus   0:0       118258.783335   12019.46   
                    cgw-tbi01      0:0                 NaN  147246.23   
                    debug          0:0                 NaN    2039.92   
                    maxwell        0:0        42041.090454    7591.27   
                    nogpfs         0:0        48790.335026   -2181.82   
                    pascal         0:0         8545.497413    2845.16   
                    producti

In [21]:
# jobs['MONTH'] = jobs['END'].dt.month

In [22]:
jobs.groupby('JOBID')['USEDMEM'].mean()

JOBID
30796171       243216.12
30853133        20604.62
30858137        57553.77
30935078        20577.96
31364111_10      9697.47
                 ...    
32002391_95      1638.59
32002391_96      1682.93
32002391_97      1685.29
32002391_98      1689.44
32002391_99      1695.11
Name: USEDMEM, Length: 99993, dtype: float64

In [23]:
jobs[jobs['JOBID'].str.contains('30853133')]

,JOBID,STATE,BEGIN,END,REQMEM,Mc_Mn,USEDMEM,DIFFMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,262144,Mn,20604.62,241539.38,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
